In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [87]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
from collections import OrderedDict, namedtuple
from sqlalchemy import func, distinct, text, and_
import pandas as pd
from IPython.display import display, HTML, Markdown
import seaborn as sns

import util
from util.year import year as current_year
from db import session
import model

pd.options.display.float_format = '{:.2f}'.format

In [114]:
evaluations = session.query(
    model.Task,
    func.count(distinct(model.User.id)).label('evals_count'),
).\
    join(model.Task.r_wave).filter(model.Wave.year == current_year.id).\
    join(model.Task.modules).\
    join(model.Module.evaluations).\
    join(model.Evaluation.r_user).\
    filter(model.User.role == 'participant')

evaluations_per_task = evaluations.\
    group_by(model.Task).order_by(model.Wave.id, model.Task.id)

evaluations_per_task_d = {
    task: evals_count
    for (task, evals_count) in evaluations_per_task.all()
}

In [115]:
successful_evaluations = evaluations_per_task.\
    filter(model.Evaluation.ok == True)

successful_evaluations_d = {
    task: evals_count
    for (task, evals_count) in successful_evaluations.all()
}

In [116]:
per_module = session.query(
    model.Evaluation.user.label('user_id'),
    func.count(model.Evaluation.id).label('eval_count'),
).\
    join(model.Evaluation.r_module).join(model.Module.r_task).\
    join(model.Task.r_wave).\
    filter(model.Wave.year == current_year.id).\
    group_by(model.Evaluation.user,
             model.Evaluation.module).subquery()

EVAL_LIMITS = [10, 30, 50]
problematic_tasks = {
    limit: {
                task: evals_count
                for (task, evals_count) in (
                    evaluations.
                    filter(model.Evaluation.ok == False).
                    join(per_module, per_module.c.user_id == model.User.id).\
                    group_by(model.Task).\
                    filter(per_module.c.eval_count > limit).all()
                )
        }
    for limit in EVAL_LIMITS
}

In [117]:
all_waves = session.query(model.Wave).\
    filter(model.Wave.year == current_year.id).\
    order_by(model.Wave.index).all()

In [119]:
def show_wave_stats(wave):
    tasks = [
        (
            task,
            evaluations_per_task_d[task],
            successful_evaluations_d[task],
            successful_evaluations_d[task] / evaluations_per_task_d[task],
            evaluations_per_task_d[task]-successful_evaluations_d[task],
            problematic_tasks[10][task] if task in problematic_tasks[10] else 0,
            problematic_tasks[30][task] if task in problematic_tasks[30] else 0,
            problematic_tasks[50][task] if task in problematic_tasks[50] else 0,
        )
        for task in evaluations_per_task_d
        if task.wave == wave.id
    ]
    tasks.sort(key=lambda x: x[0].id)

    df = pd.DataFrame(tasks, columns=[
        'Task',
        'All Evaluations',
        'Successful Evaluations',
        'Successful/All ratio',
        'N.O. Users failing on task now',
        'N.O. Users with more than 10 unsucc. evaluations',
        'N.O. Users with more than 30 unsucc. evaluations',
        'N.O. Users with more than 50 unsucc. evaluations',
    ])
    df = df.set_index('Task')
    
    cm = sns.light_palette("red", as_cmap=True)
    s = df.style.background_gradient(subset=[
        'N.O. Users failing on task now',
        'N.O. Users with more than 10 unsucc. evaluations',
        'N.O. Users with more than 30 unsucc. evaluations',
        'N.O. Users with more than 50 unsucc. evaluations',
    ], cmap=cm)
    
    cm = sns.light_palette("green", as_cmap=True)
    s.background_gradient(subset=[
        'Successful/All ratio',
    ], cmap=cm)
    
    s.bar(subset=['All Evaluations'], color='#5fd65f')
    
    display(Markdown('# {name}'.format(name=wave.caption)))
    display(s)


for wave in all_waves:
    show_wave_stats(wave)

# Nultá vlna

,All Evaluations,Successful Evaluations,Successful/All ratio,N.O. Users failing on task now,N.O. Users with more than 10 unsucc. evaluations,N.O. Users with more than 30 unsucc. evaluations,N.O. Users with more than 50 unsucc. evaluations
Task,,,,,,,
Vítej v KSI,680,680,1,0,99,17,3
Vaření proměnných,508,497,0.978346,11,189,29,7
Seznámení s pythonem,438,413,0.942922,25,132,24,6
Složitost,420,411,0.978571,9,120,20,5
Korektnost,397,369,0.929471,28,134,25,5
Úvod do seznamů,350,340,0.971429,10,199,41,14
Funkce,285,267,0.936842,18,89,19,5
Start pro zkušené,428,427,0.997664,1,11,0,0
Asymptotická složitost,288,224,0.777778,64,170,39,13


# Hrubá síla

,All Evaluations,Successful Evaluations,Successful/All ratio,N.O. Users failing on task now,N.O. Users with more than 10 unsucc. evaluations,N.O. Users with more than 30 unsucc. evaluations,N.O. Users with more than 50 unsucc. evaluations
Task,,,,,,,
Rekurze,240,240,1,0,82,26,9
Brute-force,240,239,0.995833,1,38,6,1
Permutace,214,208,0.971963,6,78,20,7
Variace s opakováním,230,229,0.995652,1,137,38,11
Variace bez opakování,196,193,0.984694,3,53,12,5
Generování podmnožin,179,174,0.972067,5,103,28,10
Backtracking,145,142,0.97931,3,30,10,3
Problém osmi dam,100,96,0.96,4,26,11,2
Jigsaw Sudoku,54,54,1,0,0,0,0


# Rozděl a panuj

,All Evaluations,Successful Evaluations,Successful/All ratio,N.O. Users failing on task now,N.O. Users with more than 10 unsucc. evaluations,N.O. Users with more than 30 unsucc. evaluations,N.O. Users with more than 50 unsucc. evaluations
Task,,,,,,,
Zmenši a panuj,151,150,0.993377,1,3,2,0
Transformuj a panuj,115,106,0.921739,9,54,18,6
Rozděl a panuj,122,121,0.991803,1,93,35,12
Binární vyhledávání,109,108,0.990826,1,62,23,9
Rozděluj a slučuj,60,60,1,0,0,0,0
QuickSort,107,107,1,0,62,26,9
Rekurzivní volání,103,101,0.980583,2,90,37,12
Binárne vyhľadávanie 2.0,76,76,1,0,0,0,0
Karlík jede na soustředění,72,72,1,0,0,0,0


# Dynamické programování

,All Evaluations,Successful Evaluations,Successful/All ratio,N.O. Users failing on task now,N.O. Users with more than 10 unsucc. evaluations,N.O. Users with more than 30 unsucc. evaluations,N.O. Users with more than 50 unsucc. evaluations
Task,,,,,,,
Úvod do dynamického programování,101,97,0.960396,4,43,19,6
Překrývající se problémy,86,66,0.767442,20,77,37,12
Bottom-up,68,68,1,0,21,13,5
Rekurentné vzťahy,66,66,1,0,35,18,6
Rekonstrukce řešení,56,53,0.946429,3,27,15,6
Nejdelší rostoucí podposloupnost,52,51,0.980769,1,49,29,9
Batoh,52,52,1,0,42,23,7
Mince,49,46,0.938776,3,26,15,5
LosKarlosovské součty,38,36,0.947368,2,17,9,3


# NP-těžká vlna

,All Evaluations,Successful Evaluations,Successful/All ratio,N.O. Users failing on task now,N.O. Users with more than 10 unsucc. evaluations,N.O. Users with more than 30 unsucc. evaluations,N.O. Users with more than 50 unsucc. evaluations
Task,,,,,,,
Opakování složitosti,73,73,1,0,54,29,11
Složitostní třídy,60,60,1,0,45,24,10
NP aneb ověřování řešení,52,50,0.961538,2,35,21,7
Redukce,68,48,0.705882,20,64,32,9
Trénink redukcí,45,45,1,0,7,4,2
O NP-úplných problémoch,40,37,0.925,3,28,19,5
NP hra a řešení složitých problémů,36,36,1,0,19,13,4
Strategie pro NP hru,23,19,0.826087,4,19,11,3
Tvorba karty,29,29,1,0,0,0,0
